# Урок 2. Создание признакового пространства.

Продолжим обработку данных с Твиттера. 

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
-	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
-	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
-	Исключим стоп-слова с помощью stop_words='english'.
-	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
-	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
-	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
-	Исключим стоп-слова с помощью stop_words='english'.
-	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().
3. Натренируем gensim.models.Word2Vec модель на наших данных.
-	Тренировать будем на токенизированных твитах combine_df['tweet_token']
-	Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
-	Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.

4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.<br>
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).

In [1]:
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [2]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [3]:
data = pd.read_pickle("lesson_01.pkl")
data.head(2)

,id,label,tweet,prep_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[father, dysfunct, selfish, drag, kid, dysfunct, run]","[father, dysfunctional, selfish, drag, kid, dysfunction, run]"
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, not, use, cause, they, do, not, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]","[thank, lyft, credit, use, caus, offer, wheelchair, van, pdx, disapoint, getthank]","[thank, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]"


1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
-	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
-	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
-	Исключим стоп-слова с помощью stop_words='english'.
-	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

In [102]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(stop_words='english', max_df=0.9, max_features=1000)

In [103]:
bag_of_words_stemmed = count_vectorizer.fit_transform(data['tweet_stemmed'].apply(lambda x: ' '.join(x)))
feature_names_stemmed = count_vectorizer.get_feature_names()
df_stemmed = pd.DataFrame(bag_of_words_stemmed.toarray(), columns = feature_names_stemmed)
df_stemmed.head(2)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,add,adventur,affirm,afternoon,age,ago,agre,ahead,aist,album,aliv,allahsoil,allow,alon,alreadi,altwaystoh,alway,amaz,america,american,angel,anger,angri,anim,anniversari,announc,anoth,answer,anti,anxieti,anymor,anyon,anyth,app,appl,appreci,arriv,ask,ass,attack,august,avail,award,away,awesom,aww,babi,bad,bag,balanc,ball,bar,bc,bday,beach,bear,beat,beauti,becom,bed,beer,begin,believ,benefit,best,bestfriend,besti,better,big,bigot,bihday,bike,bing,bird,bit,bitch,black,blame,bless,blm,block,blog,blogger,blond,blue,blur,board,bodi,bong,book,bore,born,bought,box,boy,boyfriend,brand,break,breakfast,brexit,bride,bring,bro,broken,brother,buffalo,build,bull,busi,buy,cake,calm,came,camp,campaign,cantwait,car,card,care,case,cat,caus,celebr,challeng,chanc,chang,chase,check,cheer,child,children,chill,choic,choos,christian,christina,christma,church,citi,claim,class,clean,client,climb,close,cloth,club,coach,coffe,cold,collect,colleg,color,colour,come,comment,commun,compani,complet,conc,confer,confid,confus,congrat,congratul,connect,content,continu,control,cook,cool,count,countdown,countri,coupl,cours,cover,crazi,creat,creativ,cri,cultur,cup,current,custom,cut,cute,da,dad,daddi,daili,damn,danc,dark,date,daughter,day,dead,deal,dear,death,decid,deep,definit,delet,deletetweet,depress,deserv,design,despit,develop,die,diet,differ,dinner,direct,disappoint,disgust,disney,dj,dog,domin,donald,dont,draw,dream,dress,drink,drive,drop,eah,earli,easi,eat,edit,educ,elect,els,em,email,emot,empti,end,energi,england,enjoy,environ,episod,equal,especi,eur,euro,event,everi,everybodi,everyday,everyon,everyth,ex,exactli,exam,excit,expans,expect,experi,eye,face,facebook,fact,fail,faith,fake,fall,famili,fan,fantast,far,fashion,fast,father,fathersday,favorit,favourit,fear,feel,femal,festiv,ff,fight,film,final,finger,finish,fit,flag,fli,flight,florida,flower,folk,follow,followm,food,foodi,footbal,forev,forex,forget,forward,franc,free,freedom,fresh,friday,fridayfeel,friend,friendship,fuck,fun,funni,futur,game,garden,gave,gay,gbp,gener,germani,gif,gift,girl,girlfriend,glad,glass,goal,god,goe,gold,gon,gone,good,goodmorn,goodnight,goodtim,goodvib,gop,gorgeou,gorilla,got,grace,graduat,grate,gratitud,great,green,group,grow,guess,gun,guy,gym,haha,hair,half,hand,happen,happi,happier,hard,hardcor,hate,hatr,hea,heabreak,heabroken,head,heal,health,healthi,hear,heard,hell,hello,help,hero,hey,hi,high,hill,hillari,histori,hit,hold,holiday,home,hope,horni,horribl,hot,hotel,hour,hous,hr,ht,htt,hu,hug,huge,human,hungri,husband,ibiza,ice,idea,idiot,ignor,im,imagin,impoant,impress,india,inshot,insid,inspir,instadaili,instagood,instagram,instalik,instamood,instead,internet,islam,issu,japan,job,join,joke,journey,joy,juli,june,key,kick,kid,kill,kind,king,kinki,kiss,knew,know,la,ladi,land,late,later,latest,laugh,launch,law,lawofattract,lead,leader,leadership,leakag,learn,leav,left,let,level,lgbt,liber,libtard,lie,life,lifestyl,light,lighttherapi,like,line,link,list,listen,liter,littl,live,local,lol,london,lone,long,look,lose,loser,loss,lost,lot,loud,love,loveit,lover,low,luck,lucki,lunch,mad,magic,make,makeup,man,manag,mani,market,marri,marriag,mass,match,materi,matter,mayb,mean,media,meet,melancholi,member,meme,memori,men,messag,met,miami,million,min,mind,mindset,minut,miss,model,mom,moment,monday,money,month,mood,morn,mother,motiv,mountain,movi,mr,murder,music,muslim,na,nail,nake,nasti,nation,natur,naughti,nba,nbafin,nd,need,nervou,new,news,newyork,nice,nigga,night,nobodi,nofilt,noth,notic,nude,number,nyc,obama,offer,offic,offici,oh,oil,oitnb,ok,okay,old,omg,onlin,ootd,open,orang,order,organ,origin,orlando,orlandoshoot,outsid,pa,pack,page,pain,paint,parent,pari,park,pass,passion,past,pathet,pay,peac,peopl,perfect,perform,person,pet,phone,photo,photographi,photooftheday,pic,pick,picoftheday,pictur,pig,pink,pizza,place,plan,play,player,pleas,pm,poetri,point,polar,polic,polit,politician,pool,poor,pop,porn,posit,possibl,post,power,pray,p

In [104]:
bag_of_words_lemmatized = count_vectorizer.fit_transform(data['tweet_lemmatized'].apply(lambda x: ' '.join(x)))
feature_names_lemmatized = count_vectorizer.get_feature_names()
df_lemmatized = pd.DataFrame(bag_of_words_lemmatized.toarray(), columns = feature_names_lemmatized)
df_lemmatized.head(2)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,adventure,affirmation,affirmations,afternoon,age,ago,agree,ahead,aist,album,alive,allahsoil,allow,altwaystoheal,amaze,america,american,americans,anger,angry,animals,anniversary,announce,answer,anti,anxiety,anymore,app,apple,appreciate,arrive,ask,ass,attack,august,available,award,away,awesome,aww,baby,bad,bag,balance,ball,bank,bar,bc,bday,beach,bear,beat,beautiful,beauty,bed,beer,begin,behappy,believe,benefit,best,better,big,bihday,bike,bing,bird,bitch,bite,black,blame,bless,blm,block,blog,blogger,blonde,blue,blur,board,body,bong,book,bore,box,boy,boyfriend,boys,brand,break,breakfast,brexit,bride,bring,bro,brother,brown,buffalo,build,bull,business,busy,buy,cake,calm,camp,campaign,cantwait,car,card,care,case,cat,catch,cause,cavs,celebrate,celebration,challenge,chance,change,chase,check,cheer,child,children,chill,chocolate,choice,choose,christina,christmas,church,city,claim,class,clean,click,climb,close,clothe,club,coach,cock,coffee,cold,college,color,come,comedy,comment,community,company,complete,conce,conference,confuse,congrats,congratulations,content,continue,control,cook,cool,count,countdown,country,couple,course,cover,crazy,create,creative,culture,cup,customer,customers,cut,cute,da,dad,daddy,dads,daily,damn,dance,dark,date,daughter,day,days,dead,deal,dear,death,decide,deep,delete,deletetweets,delicious,depress,depression,deserve,design,despite,development,die,diet,different,dinner,direct,disappoint,disgust,disney,dj,dog,dominate,donald,dont,download,draw,dream,dress,drink,drive,drop,eah,early,easy,eat,education,election,em,email,emotional,emotions,end,energy,england,enjoy,environment,episode,especially,essentialoils,eur,euro,euros,event,events,everybody,everyday,evil,ex,exactly,excite,exercise,expanse,expect,experience,eye,face,facebook,fact,fail,faith,fake,fall,families,family,fan,fantastic,far,fashion,fast,father,fathersday,favorite,favourite,fear,feel,female,festival,ff,fight,film,final,finally,finger,finish,fit,fitness,fix,flag,flight,florida,flower,fly,focus,folks,follow,followers,followme,food,foodie,football,forever,forex,forget,forward,france,free,freedom,fresh,friday,fridayfeeling,friend,friends,friendship,fuck,fun,funny,future,game,garden,gay,gbp,germany,gif,gift,girl,girlfriend,girls,glad,glass,goals,god,gold,gon,good,goodmorning,goodnight,goodtimes,goodvibes,gop,gorgeous,gorilla,graduation,grateful,gratitude,great,green,group,grow,guess,gun,guy,gym,haha,hair,half,hand,hang,happen,happier,happiness,happy,hard,hardcore,hat,hate,hatred,hea,heabroken,head,heal,health,healthy,hear,hell,hello,help,hero,hey,hi,high,hill,hillary,history,hit,hold,holiday,home,hop,hope,horny,horrible,hot,hotel,hour,hours,house,hrs,ht,htt,hu,hug,huge,human,humpday,hungry,husband,ibiza,ice,idea,ignorance,ignore,im,imagine,impoant,india,inshot,inside,inspiration,inspirational,inspire,instadaily,instagood,instagram,instalike,instamood,instead,internet,islam,issue,japan,job,join,joke,journey,joy,july,june,key,kick,kid,kill,kind,king,kinky,kiss,know,la,lack,ladies,lady,land,late,later,latest,laugh,launch,law,lawofattraction,lead,leadership,leakage,learn,leave,let,level,lgbt,liberal,libtard,lie,life,lifestyle,light,lighttherapy,like,line,link,list,listen,literally,little,live,local,lol,london,lonely,long,longer,look,lose,loss,lot,loud,love,loveit,lovelife,lovely,lover,low,luck,lucky,lunch,lyric,mad,magic,make,makeup,man,market,marriage,marry,mass,match,matter,maybe,mean,media,meet,melancholy,memories,men,message,miami,mind,mindset,minute,minutes,miss,model,mom,moment,moments,monday,money,month,months,mood,morning,mother,motivate,motivation,mountains,movie,movies,mr,murder,music,muslim,na,nail,naked,nasty,national,nature,naughty,nba,nbafinals,nd,near,need,nervous,ness,netflix,new,news,newyork,nice,night,nofilter,notice,nude,number,nyc,obama,offer,office,official,officially,oh,oil,oitnb,ok,okay,old,omg,ones,online,ootd,open,orange,order,organizations,orlando,orlandoshooting,outsid

2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
-	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
-	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
-	Исключим стоп-слова с помощью stop_words='english'.
-	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

In [106]:
from sklearn.feature_extraction.text import TfidfVectorizer

bag_of_words_stemmed = tfidf_vectorizer.fit_transform(data['tweet_stemmed'].apply(lambda x: ' '.join(x)))
feature_names_stemmed = tfidf_vectorizer.get_feature_names()
df_stemmed = pd.DataFrame(bag_of_words_stemmed.toarray(), columns = feature_names_stemmed)
df_stemmed.head(2)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,add,adventur,affirm,afternoon,age,ago,agre,ahead,aist,album,aliv,allahsoil,allow,alon,alreadi,altwaystoh,alway,amaz,america,american,angel,anger,angri,anim,anniversari,announc,anoth,answer,anti,anxieti,anymor,anyon,anyth,app,appl,appreci,arriv,ask,ass,attack,august,avail,award,away,awesom,aww,babi,bad,bag,balanc,ball,bar,bc,bday,beach,bear,beat,beauti,becom,bed,beer,begin,believ,benefit,best,bestfriend,besti,better,big,bigot,bihday,bike,bing,bird,bit,bitch,black,blame,bless,blm,block,blog,blogger,blond,blue,blur,board,bodi,bong,book,bore,born,bought,box,boy,boyfriend,brand,break,breakfast,brexit,bride,bring,bro,broken,brother,buffalo,build,bull,busi,buy,cake,calm,came,camp,campaign,cantwait,car,card,care,case,cat,caus,celebr,challeng,chanc,chang,chase,check,cheer,child,children,chill,choic,choos,christian,christina,christma,church,citi,claim,class,clean,client,climb,close,cloth,club,coach,coffe,cold,collect,colleg,color,colour,come,comment,commun,compani,complet,conc,confer,confid,confus,congrat,congratul,connect,content,continu,control,cook,cool,count,countdown,countri,coupl,cours,cover,crazi,creat,creativ,cri,cultur,cup,current,custom,cut,cute,da,dad,daddi,daili,damn,danc,dark,date,daughter,day,dead,deal,dear,death,decid,deep,definit,delet,deletetweet,depress,deserv,design,despit,develop,die,diet,differ,dinner,direct,disappoint,disgust,disney,dj,dog,domin,donald,dont,draw,dream,dress,drink,drive,drop,eah,earli,easi,eat,edit,educ,elect,els,em,email,emot,empti,end,energi,england,enjoy,environ,episod,equal,especi,eur,euro,event,everi,everybodi,everyday,everyon,everyth,ex,exactli,exam,excit,expans,expect,experi,eye,face,facebook,fact,fail,faith,fake,fall,famili,fan,fantast,far,fashion,fast,father,fathersday,favorit,favourit,fear,feel,femal,festiv,ff,fight,film,final,finger,finish,fit,flag,fli,flight,florida,flower,folk,follow,followm,food,foodi,footbal,forev,forex,forget,forward,franc,free,freedom,fresh,friday,fridayfeel,friend,friendship,fuck,fun,funni,futur,game,garden,gave,gay,gbp,gener,germani,gif,gift,girl,girlfriend,glad,glass,goal,god,goe,gold,gon,gone,good,goodmorn,goodnight,goodtim,goodvib,gop,gorgeou,gorilla,got,grace,graduat,grate,gratitud,great,green,group,grow,guess,gun,guy,gym,haha,hair,half,hand,happen,happi,happier,hard,hardcor,hate,hatr,hea,heabreak,heabroken,head,heal,health,healthi,hear,heard,hell,hello,help,hero,hey,hi,high,hill,hillari,histori,hit,hold,holiday,home,hope,horni,horribl,hot,hotel,hour,hous,hr,ht,htt,hu,hug,huge,human,hungri,husband,ibiza,ice,idea,idiot,ignor,im,imagin,impoant,impress,india,inshot,insid,inspir,instadaili,instagood,instagram,instalik,instamood,instead,internet,islam,issu,japan,job,join,joke,journey,joy,juli,june,key,kick,kid,kill,kind,king,kinki,kiss,knew,know,la,ladi,land,late,later,latest,laugh,launch,law,lawofattract,lead,leader,leadership,leakag,learn,leav,left,let,level,lgbt,liber,libtard,lie,life,lifestyl,light,lighttherapi,like,line,link,list,listen,liter,littl,live,local,lol,london,lone,long,look,lose,loser,loss,lost,lot,loud,love,loveit,lover,low,luck,lucki,lunch,mad,magic,make,makeup,man,manag,mani,market,marri,marriag,mass,match,materi,matter,mayb,mean,media,meet,melancholi,member,meme,memori,men,messag,met,miami,million,min,mind,mindset,minut,miss,model,mom,moment,monday,money,month,mood,morn,mother,motiv,mountain,movi,mr,murder,music,muslim,na,nail,nake,nasti,nation,natur,naughti,nba,nbafin,nd,need,nervou,new,news,newyork,nice,nigga,night,nobodi,nofilt,noth,notic,nude,number,nyc,obama,offer,offic,offici,oh,oil,oitnb,ok,okay,old,omg,onlin,ootd,open,orang,order,organ,origin,orlando,orlandoshoot,outsid,pa,pack,page,pain,paint,parent,pari,park,pass,passion,past,pathet,pay,peac,peopl,perfect,perform,person,pet,phone,photo,photographi,photooftheday,pic,pick,picoftheday,pictur,pig,pink,pizza,place,plan,play,player,pleas,pm,poetri,point,polar,polic,polit,politician,pool,poor,pop,porn,posit,possibl,post,power,pray,p

In [105]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9, max_features=1000)

In [106]:
bag_of_words_stemmed = tfidf_vectorizer.fit_transform(data['tweet_stemmed'].apply(lambda x: ' '.join(x)))
feature_names_stemmed = tfidf_vectorizer.get_feature_names()
df_stemmed = pd.DataFrame(bag_of_words_stemmed.toarray(), columns = feature_names_stemmed)
df_stemmed.head(2)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,add,adventur,affirm,afternoon,age,ago,agre,ahead,aist,album,aliv,allahsoil,allow,alon,alreadi,altwaystoh,alway,amaz,america,american,angel,anger,angri,anim,anniversari,announc,anoth,answer,anti,anxieti,anymor,anyon,anyth,app,appl,appreci,arriv,ask,ass,attack,august,avail,award,away,awesom,aww,babi,bad,bag,balanc,ball,bar,bc,bday,beach,bear,beat,beauti,becom,bed,beer,begin,believ,benefit,best,bestfriend,besti,better,big,bigot,bihday,bike,bing,bird,bit,bitch,black,blame,bless,blm,block,blog,blogger,blond,blue,blur,board,bodi,bong,book,bore,born,bought,box,boy,boyfriend,brand,break,breakfast,brexit,bride,bring,bro,broken,brother,buffalo,build,bull,busi,buy,cake,calm,came,camp,campaign,cantwait,car,card,care,case,cat,caus,celebr,challeng,chanc,chang,chase,check,cheer,child,children,chill,choic,choos,christian,christina,christma,church,citi,claim,class,clean,client,climb,close,cloth,club,coach,coffe,cold,collect,colleg,color,colour,come,comment,commun,compani,complet,conc,confer,confid,confus,congrat,congratul,connect,content,continu,control,cook,cool,count,countdown,countri,coupl,cours,cover,crazi,creat,creativ,cri,cultur,cup,current,custom,cut,cute,da,dad,daddi,daili,damn,danc,dark,date,daughter,day,dead,deal,dear,death,decid,deep,definit,delet,deletetweet,depress,deserv,design,despit,develop,die,diet,differ,dinner,direct,disappoint,disgust,disney,dj,dog,domin,donald,dont,draw,dream,dress,drink,drive,drop,eah,earli,easi,eat,edit,educ,elect,els,em,email,emot,empti,end,energi,england,enjoy,environ,episod,equal,especi,eur,euro,event,everi,everybodi,everyday,everyon,everyth,ex,exactli,exam,excit,expans,expect,experi,eye,face,facebook,fact,fail,faith,fake,fall,famili,fan,fantast,far,fashion,fast,father,fathersday,favorit,favourit,fear,feel,femal,festiv,ff,fight,film,final,finger,finish,fit,flag,fli,flight,florida,flower,folk,follow,followm,food,foodi,footbal,forev,forex,forget,forward,franc,free,freedom,fresh,friday,fridayfeel,friend,friendship,fuck,fun,funni,futur,game,garden,gave,gay,gbp,gener,germani,gif,gift,girl,girlfriend,glad,glass,goal,god,goe,gold,gon,gone,good,goodmorn,goodnight,goodtim,goodvib,gop,gorgeou,gorilla,got,grace,graduat,grate,gratitud,great,green,group,grow,guess,gun,guy,gym,haha,hair,half,hand,happen,happi,happier,hard,hardcor,hate,hatr,hea,heabreak,heabroken,head,heal,health,healthi,hear,heard,hell,hello,help,hero,hey,hi,high,hill,hillari,histori,hit,hold,holiday,home,hope,horni,horribl,hot,hotel,hour,hous,hr,ht,htt,hu,hug,huge,human,hungri,husband,ibiza,ice,idea,idiot,ignor,im,imagin,impoant,impress,india,inshot,insid,inspir,instadaili,instagood,instagram,instalik,instamood,instead,internet,islam,issu,japan,job,join,joke,journey,joy,juli,june,key,kick,kid,kill,kind,king,kinki,kiss,knew,know,la,ladi,land,late,later,latest,laugh,launch,law,lawofattract,lead,leader,leadership,leakag,learn,leav,left,let,level,lgbt,liber,libtard,lie,life,lifestyl,light,lighttherapi,like,line,link,list,listen,liter,littl,live,local,lol,london,lone,long,look,lose,loser,loss,lost,lot,loud,love,loveit,lover,low,luck,lucki,lunch,mad,magic,make,makeup,man,manag,mani,market,marri,marriag,mass,match,materi,matter,mayb,mean,media,meet,melancholi,member,meme,memori,men,messag,met,miami,million,min,mind,mindset,minut,miss,model,mom,moment,monday,money,month,mood,morn,mother,motiv,mountain,movi,mr,murder,music,muslim,na,nail,nake,nasti,nation,natur,naughti,nba,nbafin,nd,need,nervou,new,news,newyork,nice,nigga,night,nobodi,nofilt,noth,notic,nude,number,nyc,obama,offer,offic,offici,oh,oil,oitnb,ok,okay,old,omg,onlin,ootd,open,orang,order,organ,origin,orlando,orlandoshoot,outsid,pa,pack,page,pain,paint,parent,pari,park,pass,passion,past,pathet,pay,peac,peopl,perfect,perform,person,pet,phone,photo,photographi,photooftheday,pic,pick,picoftheday,pictur,pig,pink,pizza,place,plan,play,player,pleas,pm,poetri,point,polar,polic,polit,politician,pool,poor,pop,porn,posit,possibl,post,power,pray,p

In [107]:
bag_of_words_lemmatized = tfidf_vectorizer.fit_transform(data['tweet_lemmatized'].apply(lambda x: ' '.join(x)))
feature_names_lemmatized = tfidf_vectorizer.get_feature_names()
df_lemmatized = pd.DataFrame(bag_of_words_lemmatized.toarray(), columns = feature_names_lemmatized)
df_lemmatized.head(2)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,adventure,affirmation,affirmations,afternoon,age,ago,agree,ahead,aist,album,alive,allahsoil,allow,altwaystoheal,amaze,america,american,americans,anger,angry,animals,anniversary,announce,answer,anti,anxiety,anymore,app,apple,appreciate,arrive,ask,ass,attack,august,available,award,away,awesome,aww,baby,bad,bag,balance,ball,bank,bar,bc,bday,beach,bear,beat,beautiful,beauty,bed,beer,begin,behappy,believe,benefit,best,better,big,bihday,bike,bing,bird,bitch,bite,black,blame,bless,blm,block,blog,blogger,blonde,blue,blur,board,body,bong,book,bore,box,boy,boyfriend,boys,brand,break,breakfast,brexit,bride,bring,bro,brother,brown,buffalo,build,bull,business,busy,buy,cake,calm,camp,campaign,cantwait,car,card,care,case,cat,catch,cause,cavs,celebrate,celebration,challenge,chance,change,chase,check,cheer,child,children,chill,chocolate,choice,choose,christina,christmas,church,city,claim,class,clean,click,climb,close,clothe,club,coach,cock,coffee,cold,college,color,come,comedy,comment,community,company,complete,conce,conference,confuse,congrats,congratulations,content,continue,control,cook,cool,count,countdown,country,couple,course,cover,crazy,create,creative,culture,cup,customer,customers,cut,cute,da,dad,daddy,dads,daily,damn,dance,dark,date,daughter,day,days,dead,deal,dear,death,decide,deep,delete,deletetweets,delicious,depress,depression,deserve,design,despite,development,die,diet,different,dinner,direct,disappoint,disgust,disney,dj,dog,dominate,donald,dont,download,draw,dream,dress,drink,drive,drop,eah,early,easy,eat,education,election,em,email,emotional,emotions,end,energy,england,enjoy,environment,episode,especially,essentialoils,eur,euro,euros,event,events,everybody,everyday,evil,ex,exactly,excite,exercise,expanse,expect,experience,eye,face,facebook,fact,fail,faith,fake,fall,families,family,fan,fantastic,far,fashion,fast,father,fathersday,favorite,favourite,fear,feel,female,festival,ff,fight,film,final,finally,finger,finish,fit,fitness,fix,flag,flight,florida,flower,fly,focus,folks,follow,followers,followme,food,foodie,football,forever,forex,forget,forward,france,free,freedom,fresh,friday,fridayfeeling,friend,friends,friendship,fuck,fun,funny,future,game,garden,gay,gbp,germany,gif,gift,girl,girlfriend,girls,glad,glass,goals,god,gold,gon,good,goodmorning,goodnight,goodtimes,goodvibes,gop,gorgeous,gorilla,graduation,grateful,gratitude,great,green,group,grow,guess,gun,guy,gym,haha,hair,half,hand,hang,happen,happier,happiness,happy,hard,hardcore,hat,hate,hatred,hea,heabroken,head,heal,health,healthy,hear,hell,hello,help,hero,hey,hi,high,hill,hillary,history,hit,hold,holiday,home,hop,hope,horny,horrible,hot,hotel,hour,hours,house,hrs,ht,htt,hu,hug,huge,human,humpday,hungry,husband,ibiza,ice,idea,ignorance,ignore,im,imagine,impoant,india,inshot,inside,inspiration,inspirational,inspire,instadaily,instagood,instagram,instalike,instamood,instead,internet,islam,issue,japan,job,join,joke,journey,joy,july,june,key,kick,kid,kill,kind,king,kinky,kiss,know,la,lack,ladies,lady,land,late,later,latest,laugh,launch,law,lawofattraction,lead,leadership,leakage,learn,leave,let,level,lgbt,liberal,libtard,lie,life,lifestyle,light,lighttherapy,like,line,link,list,listen,literally,little,live,local,lol,london,lonely,long,longer,look,lose,loss,lot,loud,love,loveit,lovelife,lovely,lover,low,luck,lucky,lunch,lyric,mad,magic,make,makeup,man,market,marriage,marry,mass,match,matter,maybe,mean,media,meet,melancholy,memories,men,message,miami,mind,mindset,minute,minutes,miss,model,mom,moment,moments,monday,money,month,months,mood,morning,mother,motivate,motivation,mountains,movie,movies,mr,murder,music,muslim,na,nail,naked,nasty,national,nature,naughty,nba,nbafinals,nd,near,need,nervous,ness,netflix,new,news,newyork,nice,night,nofilter,notice,nude,number,nyc,obama,offer,office,official,officially,oh,oil,oitnb,ok,okay,old,omg,ones,online,ootd,open,orange,order,organizations,orlando,orlandoshooting,outsid

3. Натренируем gensim.models.Word2Vec модель на наших данных.
-	Тренировать будем на токенизированных твитах combine_df['tweet_token']
-	Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
-	Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.


In [108]:
from gensim.models import Word2Vec

sentences = data['tweet_token'].tolist()
modelW2V = Word2Vec(sentences=sentences, size=200, window=5, min_count=1, sg=1, hs=0, negative=10, workers=32, seed=34)
modelW2V.train(sentences, total_examples=len(sentences), epochs=20)

(9812014, 11728960)

4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".


In [109]:
modelW2V.most_similar(positive=['dinner'])

[('bolognese', 0.5587278008460999),
 ('eastvillage', 0.556946873664856),
 ('thegafford', 0.5521879196166992),
 ('nighout', 0.5510709881782532),
 ('spaghetti', 0.5482712388038635),
 ('roamingaround', 0.5477830171585083),
 ('applebees', 0.5420779585838318),
 ('bihdaydinner', 0.5414751172065735),
 ('marinabaysands', 0.5403186082839966),
 ('fixings', 0.5395141839981079)]

In [110]:
modelW2V.most_similar(positive=['trump'])

[('donald', 0.5754892826080322),
 ('suppoer', 0.5704171061515808),
 ('melo', 0.5407788157463074),
 ('dumptrump', 0.5397019386291504),
 ('donaldtrump', 0.5370235443115234),
 ('conman', 0.5213732719421387),
 ('fuhered', 0.519615113735199),
 ('trumptrain', 0.5179190635681152),
 ('commie', 0.5132756233215332),
 ('chopra', 0.5131602883338928)]

5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".


In [111]:
modelW2V["food"]

array([-0.5382672 , -0.822307  , -1.1076397 ,  0.20788567,  0.00676508,
       -0.9375143 , -0.12711778,  0.06241636,  0.81152886, -0.42853263,
        1.0084054 ,  0.175406  ,  0.26918113, -0.24770488,  1.1413462 ,
        0.1554971 ,  0.68073386,  0.05889169,  0.83229387, -0.08041835,
       -0.96832716,  0.08432363, -0.32383975,  0.01645085,  0.56958085,
       -0.303494  , -0.7589122 , -0.3703876 , -0.62067604, -0.2732402 ,
       -0.2704723 ,  0.46482027,  0.97524905,  0.2232971 , -0.13664782,
       -0.5302256 ,  0.23760985, -0.5536294 ,  0.07066457,  0.01670282,
       -0.08355341, -0.05977306, -0.19965322,  0.2649643 , -0.09186758,
       -1.6473475 ,  0.88990563, -0.34477141, -1.1781801 ,  0.49835122,
        0.01803767,  0.01083796,  0.68285334, -0.18955301,  0.47642782,
       -0.23895662, -0.99591714,  0.5374855 ,  0.95210946, -0.5142789 ,
       -0.20387544, -0.0648044 ,  0.20349273,  0.14138368,  0.9606067 ,
       -0.5922656 , -0.09868081,  0.3859554 ,  0.19250561, -0.61

In [112]:
modelW2V["food"].shape

(200,)

6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.<br>
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).

In [113]:
vector_w2v = np.zeros((len(sentences), 200))
for i, tweet in enumerate(data['tweet_token']):
    for t in tweet:
        vector_w2v[i] += modelW2V[t]
    l = len(tweet)
    if l > 0:
        vector_w2v[i] /= l
        
vector_w2v

array([[-0.01993618, -0.14649679, -0.18254129, ..., -0.33562695,
        -0.21970122, -0.32173526],
       [ 0.04573777, -0.22631902, -0.16654282, ..., -0.08915383,
        -0.0553489 , -0.10975735],
       [-0.10391249, -0.14971341,  0.06556412, ..., -0.35755248,
        -0.15654073, -0.36264287],
       ...,
       [-0.0934999 , -0.13452544, -0.14460867, ..., -0.26276121,
        -0.23474244, -0.32019728],
       [ 0.14748386, -0.31905348, -0.31280027, ..., -0.6464415 ,
        -0.13472368, -0.56113147],
       [ 0.15188761, -0.24955464, -0.02197799, ..., -0.22227073,
        -0.20662744, -0.34069302]])

In [114]:
vector_w2v.shape

(49159, 200)

## 2. 
Докрутить чат-бота (обучить лучший ft, w2v, взвесить idf). Модели сдампить и сложить в облако, ноутбук можно через гитхаб. чат бота разбирать будем на 3-тьем занятии, но он в материалах второго Ответы: https://yadi.sk/d/SLgpFrmrBubeYg

In [7]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

Выделяем ответы в отдельный файл

In [12]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("D:\\GB_DB\\prepared_answers.txt", "w", encoding="utf8") as fout:
    with open("D:\\GB_DB\\Otvety.txt", "r", encoding="utf8") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

Функция подготовки текста

In [13]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

Токенизация текста

In [15]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("D:\\GB_DB\\Otvety.txt", "r", encoding="utf8") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

In [16]:
sentences = [i for i in sentences if len(i) > 2]

Функция-пустышка, для возможности передать в TfidfVectorizer список уже готовых токенов, а не текст

In [48]:
def simple_tokenizer(x):
    return x

Обучение моделей 

In [49]:
tfidf_v = TfidfVectorizer(tokenizer=simple_tokenizer, lowercase=False)
tfidf_v.fit_transform(sentences)

<64662x165629 sparse matrix of type '<class 'numpy.float64'>'
	with 946117 stored elements in Compressed Sparse Row format>

In [53]:
SIZE = 200

In [54]:
modelW2V = Word2Vec(sentences=sentences, size=SIZE, window=5, min_count=2, workers=8, seed=34)
modelW2V.train(sentences, total_examples=len(sentences), epochs=50)

(52085302, 58355000)

In [55]:
modelFT = FastText(sentences=sentences, size=SIZE, min_count=2, window=5, workers=8, seed=34)
modelFT.train(sentences, total_examples=len(sentences), epochs=50)

Функция получение веса слова

In [62]:
def get_idf(word):
    ind = tfidf_v.vocabulary_.get(word)
    if ind:
        return tfidf_v.idf_[ind]
    else:
        return 1

Загружаем ответы в модель приближенного поиска

In [63]:
w2v_index = annoy.AnnoyIndex(SIZE, 'angular')
ft_index = annoy.AnnoyIndex(SIZE, 'angular')

index_map = {}
counter = 0

with open("D:\\GB_DB\\prepared_answers.txt", "r", encoding="utf8") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(SIZE)
        vector_ft = np.zeros(SIZE)
        for word in question:
            if word in modelW2V:
                vector_w2v += modelW2V[word]
                n_w2v += get_idf(word)
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += get_idf(word)
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

In [64]:
w2v_index.build(50)
ft_index.build(50)

True

Функция получения ответа на вопрос

In [65]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(SIZE)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += get_idf(word)
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

Тест

In [116]:
TEXT = "Привет!"

In [117]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Привет, тебя всегда приятно видеть!. \n',
 'ну здравствуй!. \n',
 'и вам приветик). \n']

In [118]:
get_response(TEXT, ft_index, modelFT, index_map)

['Привет, тебя всегда приятно видеть!. \n',
 'ну здравствуй!. \n',
 'и вам приветик). \n']

Сохраняем модели

In [119]:
PATH = 'D:\\GB_DB\\models\\'

with open(f'{PATH}index_map.pkl', 'wb') as f:
    pickle.dump(index_map, f)

with open(f'{PATH}tfidf_v.pkl', 'wb') as f:
    pickle.dump(tfidf_v, f)
    
w2v_index.save(f'{PATH}w2v_index')
modelW2V.save(f'{PATH}modelW2V')

ft_index.save(f'{PATH}ft_index')
modelFT.save(f'{PATH}modelFT')